In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../codes')
import pandas as pd
import numpy as np
from scipy import stats
import torch

In [2]:
df_mobiauto = pd.read_csv('../datasets/database-dashboard.csv')

In [8]:
df_mobiauto.head()

,Unnamed: 0,URL,Fabricante,Modelo,Versao,Modelo_FIPE,Valor,Cidade,Ano,Combustível,Placa final,KM,Câmbio,Cor,Carroceria
0,0,https://www.mobiauto.com.br/comprar/carros/ba-...,BMW,320i,2.0 Turbo/ActiveFlex 16V/GP 4p - G,320i 2.0 Turbo/ActiveFlex 16V/GP 4p - G,R$ 99.990,Salvador - BA,2014/2015,Flex,2,118.000,Automática,Preto,Sedan
1,1,https://www.mobiauto.com.br/comprar/carros/ba-...,Nissan,March,1.6 16V SV (Flex),March 1.6 16V SV (Flex),R$ 39.900,Salvador - BA,2013/2014,Flex,4,147.867,Manual,Cinza,Hatch
2,2,https://www.mobiauto.com.br/comprar/carros/ba-...,Volkswagen,Up!,1.0 12v TSI E-Flex Cross,Up! 1.0 12v TSI E-Flex Cross,R$ 57.900,Salvador - BA,2017/2018,Flex,9,103.139,Manual,Branco,Hatch
3,3,https://www.mobiauto.com.br/comprar/carros/ba-...,Chevrolet,Prisma,1.4 LTZ SPE/4,Prisma 1.4 LTZ SPE/4,R$ 54.900,Lauro de Freitas - BA,2017/2018,Flex,3,297.074,Manual,Vermelho,Sedan
4,4,https://www.mobiauto.com.br/comprar/carros/ba-...,Ford,Fiesta Hatch,1.6 (Flex),Fiesta Hatch 1.6 (Flex),R$ 29.900,Lauro de Freitas - BA,2012/2012,Flex,5,127.000,Manual,Branco,Hatch


In [ ]:
# testar se pesquisa por similaridade vai encontrar a versão correta
opcoes_carros = pd.read_csv('../datasets/Lista de marcas e modelos.csv')
opcoes = opcoes_carros['Modelo'].tolist()
opcoes_carros.head()

ValueError: Length of values (6546) does not match length of index (47)

In [ ]:
# comparar resultados de buscas
df_buscas = pd.DataFrame()

In [24]:
df_buscas['referencia'] = df_mobiauto['Modelo_FIPE']

In [ ]:
%pip install thefuzz[speedup]

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.7 MB/s eta 0:00:0000:0100:01


In [ ]:
from thefuzz import process
busca = df_mobiauto.iloc[0]['Modelo_FIPE']
melhor_match = process.extractOne(busca, opcoes)
print(f"String: {busca}\nMelhor match: {melhor_match[0]}; Score: {melhor_match[1]}\n")

String: 320i 2.0 Turbo/ActiveFlex 16V/GP 4p - G
Melhor match: 320iA 2.0 Turbo/ActiveFlex 16V/GP  4p; Score: 92



In [19]:
melhores_matches = df_mobiauto.apply(lambda row: process.extractOne(row['Modelo_FIPE'], opcoes)[0], axis=1)
df_buscas['thefuzz'] = melhores_matches
list(zip(df_mobiauto['Modelo_FIPE'], melhores_matches))

[('320i 2.0 Turbo/ActiveFlex 16V/GP 4p - G',
  '320iA 2.0 Turbo/ActiveFlex 16V/GP  4p'),
 ('March 1.6 16V SV (Flex)', 'MARCH SV 1.6 16V Flex Fuel'),
 ('Up! 1.0 12v TSI E-Flex Cross', 'up! cross 1.0 TSI Total Flex 12V 5p'),
 ('Prisma 1.4 LTZ SPE/4', '2300 TI/TI-4'),
 ('Fiesta Hatch 1.6 (Flex)', 'Integra GS 1.8'),
 ('Aircross 1.6 16V Shine (Flex) (Aut)',
  'AIRCROSS Shine 1.6 Flex 16V 5p Aut.'),
 ('Berlingo Furgão 1.6 16V (Flex)', 'Berlingo Furgão 1.6 16V Flex 5P'),
 ('Argo Drive 1.0 (Flex)', 'ARGO DRIVE 1.0 6V Flex'),
 ('Hilux Cabine Dupla Hilux 3.0 TDI 4x4 CD SR', 'Legend 3.2/3.5'),
 ('Sportage EX 2.0 4X2 (Flex) P586', 'Legend 3.2/3.5'),
 ('Onix 1.4 LT SPE/4', 'AM-825 Super Luxo 4.0 Diesel'),
 ('Virtus TSI 1.0', 'VIRTUS TSI 1.0 Flex 12V 4p Aut.'),
 ('Onix Plus 1.0 Premier Turbo Flex (Aut)', 'Integra GS 1.8'),
 ('Siena Essence 1.6 16V (Flex)', 'Grand Siena ESSENCE 1.6 Flex 16V'),
 ('Renegade Sport 1.8 4x2 (Aut) (Flex)',
  'Renegade Sport 1.8 4x2 Flex 16V Aut.'),
 ('320i Sport GP 2.0 Tur

In [49]:
print(process.extractOne('Ranger XLS 2.0 4x2', opcoes))
process.extractOne('Ranger XLS 4x2 2.0', opcoes)

('Ranger XLS 2.0 4x2 CD Diesel Aut. ', 90, 2057)


('NSX 3.0', 86, 2)

In [29]:
import difflib

def match_difflib(consulta):
    busca = difflib.get_close_matches(consulta, opcoes, n=1, cutoff=0.6)
    if len(busca) == 0:
        return 'Não encontrado'
    else: return busca[0]

melhores_matches = df_mobiauto.apply(lambda row: match_difflib(row['Modelo_FIPE']), axis=1)
df_buscas['difflib'] = melhores_matches
list(zip(df_mobiauto['Modelo_FIPE'], melhores_matches))

[('320i 2.0 Turbo/ActiveFlex 16V/GP 4p - G',
  '320iA 2.0 Turbo/ActiveFlex 16V/GP  4p'),
 ('March 1.6 16V SV (Flex)', 'Cerato 1.6 16V  Flex  Aut.'),
 ('Up! 1.0 12v TSI E-Flex Cross', 'Não encontrado'),
 ('Prisma 1.4 LTZ SPE/4', 'Não encontrado'),
 ('Fiesta Hatch 1.6 (Flex)', 'Fiesta SE 1.6 8V Flex 5p'),
 ('Aircross 1.6 16V Shine (Flex) (Aut)', 'Cerato 1.6 16V  Flex  Aut.'),
 ('Berlingo Furgão 1.6 16V (Flex)', 'Berlingo Furgão 1.6 16V Flex 5P'),
 ('Argo Drive 1.0 (Flex)', 'Não encontrado'),
 ('Hilux Cabine Dupla Hilux 3.0 TDI 4x4 CD SR', 'Não encontrado'),
 ('Sportage EX 2.0 4X2 (Flex) P586', 'Sportage EX 2.0 16V/ 2.0 16V Flex Aut.'),
 ('Onix 1.4 LT SPE/4', 'Não encontrado'),
 ('Virtus TSI 1.0', 'Não encontrado'),
 ('Onix Plus 1.0 Premier Turbo Flex (Aut)',
  '208 Allure 1.0 Turbo Flex 5p Aut.'),
 ('Siena Essence 1.6 16V (Flex)', 'Fiesta SE 1.6 16V Flex 5p'),
 ('Renegade Sport 1.8 4x2 (Aut) (Flex)',
  'Renegade Sport 1.8 4x2 Flex 16V Mec.'),
 ('320i Sport GP 2.0 Turbo (Aut)', '530i M Sp

Biblioteca ``thefuzz`` é superior a nativa ``difflib``, mas comete muitos erros.

In [50]:
%pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 KB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 35.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 KB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.5
    Uninstalling huggingface-hub-0.24.5:
      Successfully uninstalled huggingface-hub-0.24.5
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolve

In [52]:
%pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 743.9 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 6.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 6.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take i

In [13]:
from sentence_transformers import SentenceTransformer, util

# Carregar modelo (o 'all-MiniLM-L6-v2' é pequeno e muito bom)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Lista de textos
texts = opcoes

# Codificar os textos em embeddings
embeddings = model.encode(texts, convert_to_tensor=True)

In [15]:
# Sua consulta de busca
consulta = "Ranger XLS 4x2 2.0"

# Codificar a consulta
consulta_embedding = model.encode(consulta, convert_to_tensor=True)

# Calcular similaridade
similaridades = util.cos_sim(consulta_embedding, embeddings)

# Indice do texto mais similar
indice = torch.argmax(similaridades)
print(f"Texto mais parecido: {texts[indice]}")

Texto mais parecido: Ranger XLT 4.0 4x2 CS


In [ ]:
def melhor_match(consulta):
    # Codificar a consulta
    consulta_embedding = model.encode(consulta, convert_to_tensor=True)

    # Calcular similaridade
    similaridades = util.cos_sim(consulta_embedding, embeddings)

    # Indice do texto mais similar
    indice = torch.argmax(similaridades)
    return texts[indice]

melhores_matches = df_mobiauto.apply(lambda row: melhor_match(row['Modelo_FIPE']), axis=1)
df_buscas['sentence-transformer'] = melhores_matches
list(zip(df_mobiauto['Modelo_FIPE'], melhores_matches))

[('320i 2.0 Turbo/ActiveFlex 16V/GP 4p - G',
  '320iA 2.0 Turbo/ActiveFlex 16V/GP  4p'),
 ('March 1.6 16V SV (Flex)', 'MARCH SV 1.6 16V Flex Fuel'),
 ('Up! 1.0 12v TSI E-Flex Cross', 'up! cross 1.0 TSI Total Flex 12V 5p'),
 ('Prisma 1.4 LTZ SPE/4', 'PRISMA Sed. LTZ 1.4 8V FlexPower 4p Aut.'),
 ('Fiesta Hatch 1.6 (Flex)', 'Fiesta Sedan 1.6 16V Flex Aut.'),
 ('Aircross 1.6 16V Shine (Flex) (Aut)',
  'AIRCROSS Shine 1.6 Flex 16V 5p Aut.'),
 ('Berlingo Furgão 1.6 16V (Flex)', 'Berlingo Furgão 1.6 16V Flex 5P'),
 ('Argo Drive 1.0 (Flex)', 'ARGO DRIVE 1.0 6V Flex'),
 ('Hilux Cabine Dupla Hilux 3.0 TDI 4x4 CD SR',
  'Hilux CD SRX Limited 4x4 2.8 TDI Die.Aut'),
 ('Sportage EX 2.0 4X2 (Flex) P586', 'EcoSport XL 1.6/ 1.6 Flex 8V 5p'),
 ('Onix 1.4 LT SPE/4', 'ONIX SD. P. PR. MIDNIGHT 1.0 TB Flex Aut'),
 ('Virtus TSI 1.0', 'VIRTUS TSI 1.0 Flex 12V 4p Aut.'),
 ('Onix Plus 1.0 Premier Turbo Flex (Aut)',
  'ONIX SD. P. PR. MIDNIGHT 1.0 TB Flex Aut'),
 ('Siena Essence 1.6 16V (Flex)', 'Siena ESSENCE 1

In [30]:
df_buscas.head()

,thefuzz,difflib,setence-transformer,referencia
0,320iA 2.0 Turbo/ActiveFlex 16V/GP 4p,320iA 2.0 Turbo/ActiveFlex 16V/GP 4p,320iA 2.0 Turbo/ActiveFlex 16V/GP 4p,320i 2.0 Turbo/ActiveFlex 16V/GP 4p - G
1,MARCH SV 1.6 16V Flex Fuel,Cerato 1.6 16V Flex Aut.,MARCH SV 1.6 16V Flex Fuel,March 1.6 16V SV (Flex)
2,up! cross 1.0 TSI Total Flex 12V 5p,Não encontrado,up! cross 1.0 TSI Total Flex 12V 5p,Up! 1.0 12v TSI E-Flex Cross
3,2300 TI/TI-4,Não encontrado,PRISMA Sed. LTZ 1.4 8V FlexPower 4p Aut.,Prisma 1.4 LTZ SPE/4
4,Integra GS 1.8,Fiesta SE 1.6 8V Flex 5p,Fiesta Sedan 1.6 16V Flex Aut.,Fiesta Hatch 1.6 (Flex)


In [32]:
df_buscas.to_csv('../datasets/analise_algoritmos_de_busca.csv', index=False)

Nenhuma das alternativas foi muito boa, mas o ``sentence-transformers`` foi o melhor entre os três. Vai servir como protótipo.